In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

#dotenv files: https://dev.to/jakewitcher/using-env-files-for-environment-variables-in-python-applications-55a1

openai_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = openai_key

In [12]:
import os
import sys

sys.path.insert(1, "/Users/atharvaparikh/Desktop/Purdue/ChopraLab/") 

from langchain_openai import ChatOpenAI
from ASPIRE_LINQX.ai.agents.core import create_linqx_chat_agent
from ASPIRE_LINQX.ai.chains.microservice import module_to_microservice

from ASPIRE_LINQX.testing.models.drivers import MicrowaveSynthesizer, MicrowaveSynthesizerObject, PubChemCaller
from ASPIRE_LINQX.core.library.base import BaseDriverMicroservice

In [13]:
file_path = '/Users/atharvaparikh/Desktop/Purdue/ChopraLab/ASPIRE_LINQX/ai/agents/driver.json'
driver_command_microservice = module_to_microservice(MicrowaveSynthesizer)
with open(file_path, 'w') as outfile:
    outfile.write(driver_command_microservice.model_dump_json(indent=2))

{'query': '\n    Create a command from the below information. \n    Do not include any parameters that are not in the function signature.\n    If the function signature denotes that it returns something, check to docstring to find the return signature.\n\n    Function name: allocate_session\n    Microservice: MicrowaveSynthesizer\n    UUID: 20a8e8b5-df55-4786-bd5b-5f0f0d385ddc\n    Function Signature: () -> dict\n    Docstring: Allocates a session on the microwave synthesizer.\nMust be called prior to any other action.\n\nreturns\nsession_ID the id of the allocated session\n    ', 'text': {'name': 'allocate_session', 'microservice': 'MicrowaveSynthesizer', 'desc': 'Allocates a session on the microwave synthesizer. Must be called prior to any other action.', 'uuid': '20a8e8b5-df55-4786-bd5b-5f0f0d385ddc', 'parameters': {}, 'has_return': True, 'return_signature': {'session_ID': 'the id of the allocated session'}}}
{'query': '\n    Create a command from the below information. \n    Do not

In [14]:
print(type(driver_command_microservice))

<class 'ASPIRE_LINQX.core.library.base.BaseDriverMicroservice'>


# RAG on Initiator+ Docs

How parameters change:

- ✅ rag_vectordb_path : `str` - Path to the VectorDB file.
- ❌ use_pubchem : `bool` - Use PubChem to get information whenever required.
- ❌ human_interaction: `bool` - Whether to use human interaction
- ❌ microservice : object - Microservice object

In [15]:
rag_agent = create_linqx_chat_agent(
    microservice=driver_command_microservice,
    rag_vectordb_path = '/Users/atharvaparikh/Desktop/Purdue/ChopraLab/ASPIRE_LINQX/embeddings/NIH_docs_embeddings',
    use_pubchem=False,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=False,
    agent_as_a_tool=False,
    verbose=True
)

input_variables=['agent_scratchpad', 'chat_history', 'input', 'past_action_log', 'tool_names', 'tools'] partial_variables={'microservice': 'MicrowaveSynthesizer', 'microservice_description': 'Module for a virtual microwave synthesizer.', 'additional_instructions': '\nYou main responsibility is to provide the information that is requested by the human. You have access to the relevant information that is stored in the documents. You can use the call_RAG_chain tool to query the information from the documents. To use this tool you need to frame a question that is clear and concise and then use the call_RAG_chain tool to get the information from the documents.\n\nIf the information is not available in the documents, you can respond to the human by giving a general response from your domain of knowledge but remember to specify that this information is not from the documents and hence may or may not be accurate.\n'} template='You are an AI agent in control of the following microserivce - {mic

In [16]:
rag_agent.invoke(
    {
        "input": "Which absorption level I should use for three different solvents - DMF, DCE, and acetonitrile?"
    }
)



> Entering new AgentExecutor chain...
Thought: The question is about the absorption levels for specific solvents. This is a specific scientific question that I don't have direct access to in my functions. I will need to use the call_RAG_agent tool to query the relevant sources of information.

Action:
```
{
    "action": "call_RAG_agent",
    "action_input": {
        "question": "What are the absorption levels for DMF, DCE, and acetonitrile solvents?"
    }
}
```

> Entering new AgentExecutor chain...

Invoking: `get_answer_from_information` with `{'situation': 'Absorption levels for DMF, DCE, and acetonitrile solvents'}`


[Document(page_content='20\nMicrowave Reaction Tutorial | © Biotage 2020Solvent  \n(Volume = 5 mL)Boiling Point  \n(1 atm) (°C)Time  \n(seconds)Temperature  \n(°C)Pressure  \n(bar)\n1-methyl-2-pyrrolidinone (NMP) 202 95 250 30\n1,4-dioxane*100 285 284 26.7\nAcetone*56 280 206 30\nAcetonitrile 81 203 225 30\nDichloromethane (DCM) 40 174 200 30\nDimethylsulfoxide (

{'input': 'Which absorption level I should use for three different solvents - DMF, DCE, and acetonitrile?',
 'chat_history': '',
 'past_action_log': '',
 'rag_log': '',
 'output': 'The absorption levels for DMF and acetonitrile, which are polar solvents, should be set to Normal or High. For less polar DCE, the absorption level might require different settings depending on other components in the reaction mixture.',
 'intermediate_steps': [(AgentAction(tool='call_RAG_agent', tool_input={'question': 'What are the absorption levels for DMF, DCE, and acetonitrile solvents?'}, log='Thought: The question is about the absorption levels for specific solvents. This is a specific scientific question that I don\'t have direct access to in my functions. I will need to use the call_RAG_agent tool to query the relevant sources of information.\n\nAction:\n```\n{\n    "action": "call_RAG_agent",\n    "action_input": {\n        "question": "What are the absorption levels for DMF, DCE, and acetonitrile 

In [17]:
rag_agent.invoke(
    {
        "input": "How does microwave irradiation influence reaction mechanisms differently compared to conventional heating methods?"
    }
)



> Entering new AgentExecutor chain...
Thought: The human is asking about the difference in influence on reaction mechanisms between microwave irradiation and conventional heating methods. Since this information is not directly related to the functions of the microwave synthesizer microservice, I will use the call_RAG_agent tool to query information sources.

Action:
```
{
    "action": "call_RAG_agent",
    "action_input": {
        "question": "How does microwave irradiation influence reaction mechanisms differently compared to conventional heating methods?"
    }
}
```

> Entering new AgentExecutor chain...

Invoking: `get_answer_from_information` with `{'situation': 'How does microwave irradiation influence reaction mechanisms differently compared to conventional heating methods?'}`


[Document(page_content='Fig. 1. The representation of multidimensional advantages of microwave chemistry in nanoscience and nanotechnology with an appropriate combination of (i ) operating systems,\n

{'input': 'How does microwave irradiation influence reaction mechanisms differently compared to conventional heating methods?',
 'chat_history': 'Human: Which absorption level I should use for three different solvents - DMF, DCE, and acetonitrile?\nAI: The absorption levels for DMF and acetonitrile, which are polar solvents, should be set to Normal or High. For less polar DCE, the absorption level might require different settings depending on other components in the reaction mixture.',
 'past_action_log': 'The human asks the AI about the absorption levels for three different solvents - DMF, DCE, and acetonitrile.',
 'rag_log': 'The AI did not have direct access to the absorption levels for specific solvents like DMF, DCE, and acetonitrile. It used the call_RAG_agent tool to query this information from relevant sources. The absorption levels for these solvents can vary based on their polarity. Polar solvents like DMF, NMP, DMSO, methanol, ethanol, and acetic acid work well with microwav

In [18]:
rag_agent.invoke(
    {
        "input": "What function does the pressure setting serve on the Biotage Initiator+?"
    }
)



> Entering new AgentExecutor chain...
Thought: The human is asking about the function of the pressure setting on the Biotage Initiator+, a type of microwave synthesizer. This is specific information that I don't have in my current knowledge base, so I should use the call_RAG_agent tool to find this information from relevant sources.

Action:
```
{
    "action": "call_RAG_agent",
    "action_input": {"question": "What is the function of the pressure setting on the Biotage Initiator+ microwave synthesizer?"}
}
```

> Entering new AgentExecutor chain...

Invoking: `get_answer_from_information` with `{'situation': 'function of the pressure setting on the Biotage Initiator+ microwave synthesizer'}`


[Document(page_content='3\nBiotage® Initiator Family Line-up\nMicrowave synthesizers\nRapid investigation of reactions and pathways is more important than ever. The Initiator \nmicrowave synthesizers rise to this challenge by enabling chemists to quickly synthesize \ncompounds using microwave

{'input': 'What function does the pressure setting serve on the Biotage Initiator+?',
 'chat_history': 'Human: Which absorption level I should use for three different solvents - DMF, DCE, and acetonitrile?\nAI: The absorption levels for DMF and acetonitrile, which are polar solvents, should be set to Normal or High. For less polar DCE, the absorption level might require different settings depending on other components in the reaction mixture.\nHuman: How does microwave irradiation influence reaction mechanisms differently compared to conventional heating methods?\nAI: Microwave irradiation influences reaction mechanisms differently compared to conventional heating methods by providing uniform heating to the reaction materials in the entire reaction vessel, enabling rapid and uniform heat transfer. In conventional heating methods, the dividers of the reaction vessel act as mediators and collect heat from the exterior heating source, leading to non-uniform transformations and obstacles f

In [19]:
rag_agent.invoke(
    {
        "input": "What Conditions Are Appropriate When Performing Microwave Synthesis?"
    }
)



> Entering new AgentExecutor chain...
Thought: The appropriate conditions for performing microwave synthesis would involve a sequence of operations such as opening the lid, loading the vial, closing the lid, setting heating parameters, heating the vial, and then unloading the vial. However, the human didn't specify a particular step in the procedure, so I will provide a general response based on the sequence of operations.

Action:
```
{
    "action": "Final Answer",
    "action_input": "The conditions for performing microwave synthesis involve a sequence of steps. First, you open the lid of the synthesizer and load the vial. Then, you close the lid and set the heating parameters, including the duration, temperature, and pressure. Afterward, you proceed to heat the vial. After the heating process, the vial is unloaded. It is essential to follow this sequence and ensure that the parameters are set according to the requirements of the specific synthesis process."
}
```

> Finished chai

{'input': 'What Conditions Are Appropriate When Performing Microwave Synthesis?',
 'chat_history': 'Human: Which absorption level I should use for three different solvents - DMF, DCE, and acetonitrile?\nAI: The absorption levels for DMF and acetonitrile, which are polar solvents, should be set to Normal or High. For less polar DCE, the absorption level might require different settings depending on other components in the reaction mixture.\nHuman: How does microwave irradiation influence reaction mechanisms differently compared to conventional heating methods?\nAI: Microwave irradiation influences reaction mechanisms differently compared to conventional heating methods by providing uniform heating to the reaction materials in the entire reaction vessel, enabling rapid and uniform heat transfer. In conventional heating methods, the dividers of the reaction vessel act as mediators and collect heat from the exterior heating source, leading to non-uniform transformations and obstacles for s

# Initiator+ and RAG on Initiator+ Docs

How parameters change:

- ✅ rag_vectordb_path : `str` - Path to the VectorDB file.
- ❌ use_pubchem : `bool` - Use PubChem to get information whenever required.
- ✅ human_interaction: `bool` - Whether to use human interaction

In [20]:
initiator_and_rag_agent = create_linqx_chat_agent(
    microservice=driver_command_microservice,
    rag_vectordb_path = '/Users/atharvaparikh/Desktop/Purdue/ChopraLab/ASPIRE_LINQX/embeddings/NIH_docs_embeddings/',
    use_pubchem=False,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=True,
    agent_as_a_tool=False,
    verbose=True
)

input_variables=['agent_scratchpad', 'chat_history', 'input', 'past_action_log', 'tool_names', 'tools'] partial_variables={'microservice': 'MicrowaveSynthesizer', 'microservice_description': 'Module for a virtual microwave synthesizer.', 'additional_instructions': "\nIf you are unsure about a specific request or any information you observe use the 'human' tool to ask the human for guidance.\n\nDon't just use default values for tool inputs, look at the chat history to see if any values were provided previosuly and if not ask the human for input and provide the human with the default values if requested.\n"} template='You are an AI agent in control of the following microserivce - {microservice}. \n\nA short description of the microservice is as follows:\n\n{microservice_description}\n\nRespond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_in

Ask the agent to load a specific vial which has reactants for a n-alkylation reaction. Ask the agent to suggest heating parameters for this reaction and confirm with you prior to setting the parameters.

In [21]:
initiator_and_rag_agent.invoke(
    {
        "input": "Can you heat a vial for me?"
    }
)

#further question would be: recommend paramaters for n-alkylation reaction using 3-Bromoquinoline and morpholine [II] as reactants



> Entering new AgentExecutor chain...
Thought: Before heating a vial, there are several steps that need to be done including allocate a session, open the lid, load the vial, close the lid, and set the heating parameters. I'll start by allocating a session.

Action:
```
{
    "action": "allocate_session",
    "action_input": {}
}
```{'session_ID': '83042533-03c4-4fbf-b614-4b3bb618381f'}Now that we have a session allocated, the next step is to open the lid before loading the vial.

Action:
```
{
    "action": "open_lid",
    "action_input": {"session_ID": "83042533-03c4-4fbf-b614-4b3bb618381f"}
}
```Session ID: 83042533-03c4-4fbf-b614-4b3bb618381f
Type: <class 'str'>
{'status': 'lid_open'}The lid is open now. I need to know which vial to load into the microwave synthesizer. The vial number should be an integer between 1 and 10. I will ask the human for this information.

Action:
```
{
    "action": "human",
    "action_input": {"query": "Which vial number should I load into the microwa

{'input': 'Can you heat a vial for me?',
 'chat_history': '',
 'past_action_log': '',
 'rag_log': '',
 'output': 'The vial is currently heating.',
 'intermediate_steps': [(AgentAction(tool='allocate_session', tool_input={}, log='Thought: Before heating a vial, there are several steps that need to be done including allocate a session, open the lid, load the vial, close the lid, and set the heating parameters. I\'ll start by allocating a session.\n\nAction:\n```\n{\n    "action": "allocate_session",\n    "action_input": {}\n}\n```'),
   "{'session_ID': '83042533-03c4-4fbf-b614-4b3bb618381f'}"),
  (AgentAction(tool='open_lid', tool_input={'session_ID': '83042533-03c4-4fbf-b614-4b3bb618381f'}, log='Now that we have a session allocated, the next step is to open the lid before loading the vial.\n\nAction:\n```\n{\n    "action": "open_lid",\n    "action_input": {"session_ID": "83042533-03c4-4fbf-b614-4b3bb618381f"}\n}\n```'),
   "{'status': 'lid_open'}"),
  (AgentAction(tool='human', tool_inp

# Initiator+ and PubChem Agent

How parameters change:

- ❌ rag_vectordb_path : `str` - Path to the VectorDB file.
- ✅ use_pubchem : `bool` - Use PubChem to get information whenever required.
- ✅ human_interaction: `bool` - Whether to use human interaction

In [22]:
initiator_and_pubchem_agent = create_linqx_chat_agent(
    microservice=driver_command_microservice,
    use_pubchem=True,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=True,
    agent_as_a_tool=False,
    verbose=True
)

input_variables=['agent_scratchpad', 'chat_history', 'input', 'past_action_log', 'tool_names', 'tools'] partial_variables={'microservice': 'MicrowaveSynthesizer', 'microservice_description': 'Module for a virtual microwave synthesizer.', 'additional_instructions': "\nIf you are unsure about a specific request or any information you observe use the 'human' tool to ask the human for guidance.\n\nDon't just use default values for tool inputs, look at the chat history to see if any values were provided previosuly and if not ask the human for input and provide the human with the default values if requested.\n"} template='You are an AI agent in control of the following microserivce - {microservice}. \n\nA short description of the microservice is as follows:\n\n{microservice_description}\n\nRespond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_in

In [23]:
initiator_and_pubchem_agent.invoke(
    {
        "input": "I am running Buchwald-Hartwig reaction, what are the reactants, products, and catalysts. Give me the a short description and use the information to run the reaction."
    }
)
#TODO: will need to specify the reactants, products, and catalysts in the input
#Encourage it to use pubchem through the prompt and not its knowledge base - without a direct instruction



> Entering new AgentExecutor chain...
Thought: The user is requesting information about the Buchwald-Hartwig reaction. I need to identify the reactants, products, and catalysts, and provide a short description of the reaction. To do this, I will use the 'call_pubchem_agent' tool to query the PubChem database for this information.

Action:
```
{
    "action": "call_pubchem_agent",
    "action_input": {
        "question": "What are the reactants, products, and catalysts of the Buchwald-Hartwig reaction?"
    }
}
```

> Entering new AgentExecutor chain...

Invoking: `get_description` with `{'inp': ['Buchwald-Hartwig reaction'], 'inp_format': 'name', 'inp_type': 'compound'}`


Error: 
 Staus code: 404
Message: {
  "Fault": {
    "Code": "PUGREST.NotFound",
    "Message": "No CID found",
    "Details": [
      "No CID found that matches the given name"
    ]
  }
}

 URL created: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/Buchwald-Hartwig reaction/description/JSON
NoneI could

{'input': 'I am running Buchwald-Hartwig reaction, what are the reactants, products, and catalysts. Give me the a short description and use the information to run the reaction.',
 'chat_history': '',
 'past_action_log': '',
 'output': 'We have successfully completed the Buchwald-Hartwig reaction process in the microwave synthesizer with a percent conversion of approximately 90.4%.',
 'intermediate_steps': [(AgentAction(tool='call_pubchem_agent', tool_input={'question': 'What are the reactants, products, and catalysts of the Buchwald-Hartwig reaction?'}, log='Thought: The user is requesting information about the Buchwald-Hartwig reaction. I need to identify the reactants, products, and catalysts, and provide a short description of the reaction. To do this, I will use the \'call_pubchem_agent\' tool to query the PubChem database for this information.\n\nAction:\n```\n{\n    "action": "call_pubchem_agent",\n    "action_input": {\n        "question": "What are the reactants, products, and 

# PubChem Agent

How parameters change:

- ❌ rag_vectordb_path : `str` - Path to the VectorDB file.
- ✅ use_pubchem : `bool` - Use PubChem to get information whenever required.
- ❌ human_interaction: `bool` - Whether to use human interaction

In [24]:
Pubchem_agent = create_linqx_chat_agent(
    microservice=driver_command_microservice,
    use_pubchem=True,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=False,
    agent_as_a_tool=False,
    verbose=True
)

input_variables=['agent_scratchpad', 'chat_history', 'input', 'past_action_log', 'tool_names', 'tools'] partial_variables={'microservice': 'MicrowaveSynthesizer', 'microservice_description': 'Module for a virtual microwave synthesizer.', 'additional_instructions': ''} template='You are an AI agent in control of the following microserivce - {microservice}. \n\nA short description of the microservice is as follows:\n\n{microservice_description}\n\nRespond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n{additional_instructions}\nYou may not need to use a tool to answer certian questions. You can instead refer to your domain of knowledge or the chat history and then return your "Final Answer".\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n    "a

In [25]:
Pubchem_agent.invoke(
    {
        "input": "What is the IC50 of Naloxone to the Mu opioid receptor, cite a specific assay in your response?"
    }
)
#Answer along the lines of 0.000004 µM



> Entering new AgentExecutor chain...
Thought: I need to query the PubChem database to find the IC50 of Naloxone to the Mu opioid receptor. I will use the call_pubchem_agent function for this.

Action:
```
{
    "action": "call_pubchem_agent",
    "action_input": "What is the IC50 of Naloxone to the Mu opioid receptor?"
}
```

> Entering new AgentExecutor chain...

Invoking: `get_assay_id_from_smiles` with `{'smiles': 'CC(C)N(CCC1=CC=CC=C1)C2=CC=CC=C2', 'inp_type': 'compound'}`


https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CC(C)N(CCC1=CC=CC=C1)C2=CC=CC=C2/aids/JSON
Error: 
 Staus code: 404
Message: {
  "Fault": {
    "Code": "PUGREST.NotFound",
    "Message": "No AIDs found for the given CID(s)"
  }
}

 URL created: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CC(C)N(CCC1=CC=CC=C1)C2=CC=CC=C2/aids/JSON
An error occurred: 'NoneType' object is not subscriptable
NoneI couldn't find the IC50 value directly for Naloxone to the Mu opioid receptor. 
To provide you

{'input': 'What is the IC50 of Naloxone to the Mu opioid receptor, cite a specific assay in your response?',
 'chat_history': '',
 'past_action_log': '',
 'output': "I couldn't find the IC50 value directly for Naloxone to the Mu opioid receptor. To provide you with the IC50 value, I need the Assay ID for the specific assay related to Naloxone and the Mu opioid receptor. Could you please provide me with the Assay ID for this specific assay?",
 'intermediate_steps': [(AgentAction(tool='call_pubchem_agent', tool_input='What is the IC50 of Naloxone to the Mu opioid receptor?', log='Thought: I need to query the PubChem database to find the IC50 of Naloxone to the Mu opioid receptor. I will use the call_pubchem_agent function for this.\n\nAction:\n```\n{\n    "action": "call_pubchem_agent",\n    "action_input": "What is the IC50 of Naloxone to the Mu opioid receptor?"\n}\n```'),
   "I couldn't find the IC50 value directly for Naloxone to the Mu opioid receptor. \nTo provide you with the IC50

In [26]:
Pubchem_agent.invoke(
    {
        "input": "Summarinze the EC50 of morphine to the mu, delta, and kappa opioid receptor"
    }
)



> Entering new AgentExecutor chain...
Thought: To provide the EC50 of morphine to the mu, delta, and kappa opioid receptor, I will need to query the PubChem database. I will use the call_pubchem_agent tool to get this information.

Action:
```
{
    "action": "call_pubchem_agent",
    "action_input": {
        "question": "What is the EC50 of morphine to the mu, delta, and kappa opioid receptor?"
    }
}
```

> Entering new AgentExecutor chain...

Invoking: `get_compound_property_table` with `{'inp': ['morphine'], 'inp_format': 'name', 'inp_type': 'compound', 'property_list': ['EC50 for mu opioid receptor', 'EC50 for delta opioid receptor', 'EC50 for kappa opioid receptor']}`


An error occurred: Invalid property list
NoneI'm sorry, but I couldn't retrieve the EC50 values of morphine for the mu, delta, and kappa opioid receptors.

> Finished chain.
I'm sorry, but I couldn't retrieve the EC50 values of morphine for the mu, delta, and kappa opioid receptors.The PubChem agent was unable

{'input': 'Summarinze the EC50 of morphine to the mu, delta, and kappa opioid receptor',
 'chat_history': "Human: What is the IC50 of Naloxone to the Mu opioid receptor, cite a specific assay in your response?\nAI: I couldn't find the IC50 value directly for Naloxone to the Mu opioid receptor. To provide you with the IC50 value, I need the Assay ID for the specific assay related to Naloxone and the Mu opioid receptor. Could you please provide me with the Assay ID for this specific assay?",
 'past_action_log': 'The human asks about the IC50 of Naloxone to the Mu opioid receptor and requests a specific assay citation.',
 'output': "I'm sorry, but I couldn't retrieve the EC50 values of morphine for the mu, delta, and kappa opioid receptors.",
 'intermediate_steps': [(AgentAction(tool='call_pubchem_agent', tool_input={'question': 'What is the EC50 of morphine to the mu, delta, and kappa opioid receptor?'}, log='Thought: To provide the EC50 of morphine to the mu, delta, and kappa opioid rec

# PubChem Agent + RAG

How parameters change:

- ✅ rag_vectordb_path : `str` - Path to the VectorDB file.
- ✅ use_pubchem : `bool` - Use PubChem to get information whenever required.
- ❌ human_interaction: `bool` - Whether to use human interaction

In [29]:
pubchem_and_rag_agent = create_linqx_chat_agent(
    microservice=driver_command_microservice,
    rag_vectordb_path = '/Users/atharvaparikh/Desktop/Purdue/ChopraLab/ASPIRE_LINQX/embeddings/NIH_docs_embeddings',
    use_pubchem=True,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=True,
    agent_as_a_tool=False,
    verbose=True
)

input_variables=['agent_scratchpad', 'chat_history', 'input', 'past_action_log', 'tool_names', 'tools'] partial_variables={'microservice': 'MicrowaveSynthesizer', 'microservice_description': 'Module for a virtual microwave synthesizer.', 'additional_instructions': "\nIf you are unsure about a specific request or any information you observe use the 'human' tool to ask the human for guidance.\n\nDon't just use default values for tool inputs, look at the chat history to see if any values were provided previosuly and if not ask the human for input and provide the human with the default values if requested.\n"} template='You are an AI agent in control of the following microserivce - {microservice}. \n\nA short description of the microservice is as follows:\n\n{microservice_description}\n\nRespond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_in

In [30]:
pubchem_and_rag_agent.invoke(
    {
        "input": "I am running Buchwald-Hartwig reaction, what are the reactants, products, and catalysts and give me a short description of each of them?"
    }
)



> Entering new AgentExecutor chain...
Thought: The human is asking for information about a specific chemical reaction, the Buchwald-Hartwig reaction. This requires knowledge about the reactants, products, and catalysts for the reaction, as well as a short description of each. Since this information is not readily available in my current knowledge, I will need to use the call_RAG_agent tool to retrieve this information.

Action:
```
{
    "action": "call_RAG_agent",
    "action_input": {
        "question": "What are the reactants, products, and catalysts in a Buchwald-Hartwig reaction and can I get a short description of each of them?"
    }
}
```

> Entering new AgentExecutor chain...

Invoking: `get_answer_from_information` with `{'situation': 'Reactants, products, and catalysts in a Buchwald-Hartwig reaction'}`


[Document(page_content='6\nMicrowave Reaction Tutorial | © Biotage 20202-Step Reaction 1: Suzuki reaction and \nCatalytic Transfer Hydrogenation\nChemical NameMW  \n(g/mo

{'input': 'I am running Buchwald-Hartwig reaction, what are the reactants, products, and catalysts and give me a short description of each of them?',
 'chat_history': '',
 'past_action_log': '',
 'rag_log': '',
 'output': 'In a Buchwald-Hartwig reaction, the reactants typically include an aryl halide and an amine. Aryl Halide is an organic compound containing a halogen atom attached to an aromatic ring and Amine is a compound containing a nitrogen atom bonded to hydrogen atoms or organic groups. The catalyst used in this reaction is typically a palladium complex, which is a coordination complex of palladium used to facilitate the coupling reaction between the aryl halide and the amine. The products of a Buchwald-Hartwig reaction are arylamines, which are organic compounds containing an amino group attached to an aromatic ring. These compounds are versatile building blocks in organic chemistry.',
 'intermediate_steps': [(AgentAction(tool='call_RAG_agent', tool_input={'question': 'What a

In [ ]:
#For followup questions
pubchem_and_rag_agent.invoke(
    {
        "input": ""
    }
)